In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import random
from sklearn import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from scipy.sparse import hstack, vstack, csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import lightgbm as lgb
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import punkt

import string
import re
import gc
#from plotly import plotly
#import plotly.offline as offline
#import plotly.graph_objs as go
#offline.init_notebook_mode()
#from collections import Counter

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence, text
from keras.layers import Input, Embedding

In [0]:
import pylab as pl # linear algebra + plots
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import gc
#import xgboost as xgb
#import lightgbm as lgb
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict, Counter
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
#from textblob import TextBlob
from scipy.stats import pearsonr
from scipy.sparse import hstack
from multiprocessing import Pool

In [0]:
X = pd.DataFrame(pd.read_pickle('/content/gdrive/Shared drives/Predictive Analytics Project/Data/TFIDF/X_train_tfidf.pkl'))

In [0]:
X_test = pd.DataFrame(pd.read_pickle('/content/gdrive/Shared drives/Predictive Analytics Project/Data/TFIDF/X_test_tfidf.pkl'))

In [0]:
y = pd.DataFrame(pd.read_pickle('/content/gdrive/Shared drives/Predictive Analytics Project/Data/TFIDF/y_train.pkl'))




In [0]:
import gc
import numpy as np
import pandas as pd
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import lightgbm as lgb

In [0]:
feature_names = list(X.columns)

for i in range(0, len(feature_names)): 
    feature_names[i] = str(feature_names[i]) 


In [22]:
test = pd.read_csv('/content/gdrive/Shared drives/Predictive Analytics Project/Data/Original Dataset/test.csv')
from google.colab import files

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import timeit
def code_to_test():

  cv_scores = []
  xgb_preds = []
  seed = 28 # Get your own seed

  K = 5  
  kf = KFold(n_splits = K, random_state = seed, shuffle = True)

  for train_index, test_index in kf.split(X):
    
    # Split out a validation set
      X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=random.seed(seed))
    
    # params are tuned with kaggle kernels in mind
      xgb_params = {'eta': 0.15, 
                  'max_depth': 7, 
                  'subsample': 0.80, 
                  'colsample_bytree': 0.80, 
                  'objective': 'binary:logistic', 
                  'eval_metric': 'auc', 
                  'seed': seed
                 }
    
      d_train = xgb.DMatrix(X_train, y_train)
      d_valid = xgb.DMatrix(X_valid, y_valid)
      d_test = xgb.DMatrix(X_test)
    
      watchlist = [(d_train, 'train'), (d_valid, 'valid')]
      model = xgb.train(xgb_params, d_train, 2000, watchlist, verbose_eval=50, early_stopping_rounds=30)
      cv_scores.append(float(model.attributes()['best_score']))
      xgb_pred = model.predict(d_test)
      xgb_preds.append(list(xgb_pred))

  del X_train, X_valid, y_train, y_valid, d_train, d_valid, d_test; gc.collect()

elapsed_time = timeit.timeit(code_to_test, number=100)/100
print(elapsed_time)

[0]	train-auc:0.659049	valid-auc:0.645413
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[50]	train-auc:0.862493	valid-auc:0.7264
[100]	train-auc:0.927837	valid-auc:0.727311
Stopping. Best iteration:
[76]	train-auc:0.902056	valid-auc:0.72832

[0]	train-auc:0.663801	valid-auc:0.643301
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[50]	train-auc:0.864305	valid-auc:0.7209
[100]	train-auc:0.926772	valid-auc:0.72492
Stopping. Best iteration:
[114]	train-auc:0.936694	valid-auc:0.725964

[0]	train-auc:0.661656	valid-auc:0.638696
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[50]	train-auc:0.865502	valid-auc:0.717075
[100]	train-auc:0.925992	valid-auc:0.722764
[150]	train-auc:0.956158	valid-auc:0.722515
Stopp

In [0]:
p = model.predict(d_test)

submission = pd.DataFrame({
    "id": test["id"],
    "project_is_approved": p.reshape(-1),
    })
submission.to_csv('xgb_1.csv', index=False)
files.download('xgb_1.csv')

In [0]:
cv_scores = []
xgb_preds = []
seed = 28 # Get your own seed

K = 5 # How many folds do you want? 
kf = KFold(n_splits = K, random_state = seed, shuffle = True)

for train_index, test_index in kf.split(X):
    
    # Split out a validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=random.seed(seed))
    
    # params are tuned with kaggle kernels in mind
    xgb_params = {'eta': 0.15, 
                  'max_depth': 7, 
                  'subsample': 0.80, 
                  'colsample_bytree': 0.80, 
                  'objective': 'binary:logistic', 
                  'eval_metric': 'auc',
                  'learning_rate': 0.025,
                  'seed': seed
                 }
    
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    d_test = xgb.DMatrix(X_test)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 2000, watchlist, verbose_eval=50, early_stopping_rounds=30)
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

p = model.predict(d_test)

submission = pd.DataFrame({
    "id": test["id"],
    "project_is_approved": p.reshape(-1),
    })
submission.to_csv('xgb_1.csv', index=False)
files.download('xgb_1.csv')
    
del X_train, X_valid, y_train, y_valid, d_train, d_valid, d_test; gc.collect()

[0]	train-auc:0.626249	valid-auc:0.616731
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[50]	train-auc:0.706305	valid-auc:0.647095
[100]	train-auc:0.734926	valid-auc:0.6527
[150]	train-auc:0.767965	valid-auc:0.657119
[200]	train-auc:0.799581	valid-auc:0.660312
[250]	train-auc:0.826082	valid-auc:0.662386
[300]	train-auc:0.850068	valid-auc:0.663328
[350]	train-auc:0.871629	valid-auc:0.663698
Stopping. Best iteration:
[335]	train-auc:0.866054	valid-auc:0.663908

[0]	train-auc:0.627623	valid-auc:0.62006
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 30 rounds.
[50]	train-auc:0.702802	valid-auc:0.648757
[100]	train-auc:0.731701	valid-auc:0.654228
[150]	train-auc:0.766195	valid-auc:0.659347
[200]	train-auc:0.797777	valid-auc:0.664245
[250]	train-auc:0.823407	valid-auc:0.66609
[300]	train-auc:0.847667	valid-auc:0.

TypeError: ignored

In [0]:
p = model.predict(d_test)

submission = pd.DataFrame({
    "id": test["id"],
    "project_is_approved": p.reshape(-1),
    })
submission.to_csv('xgb_1.csv', index=False)
files.download('xgb_1.csv')
    